<a href="https://colab.research.google.com/github/elviobg/titanic/blob/main/Titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
from google.colab import files
drive.mount('/content/drive')

import os
ROOT_PATH = '/content/drive/My Drive/Colab Notebooks/Titanic/'
DATASETS_PATH = '/content/drive/My Drive/Colab Notebooks/Titanic/dataset/'
os.chdir(ROOT_PATH)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import numpy as np

In [3]:
train_original = pd.read_csv(DATASETS_PATH+'train.csv')
test_original = pd.read_csv(DATASETS_PATH+'test.csv')
print("Data loaded...")
train_original.head()

Data loaded...


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
train_original = train_original.rename(columns = {'Survived': 'survived', 
                                                  'Pclass': 'pclass', 
                                                  'Name': 'name', 
                                                  'Sex': 'sex', 
                                                  'Age': 'age', 
                                                  'SibSp': 'sibsp',
                                                  'Parch': 'parch', 
                                                  'Ticket': 'ticket',
                                                  'Fare': 'fare', 
                                                  'Embarked': 'embarked'}, inplace = False)

test_original = test_original.rename(columns = {'Survived': 'survived', 
                                                  'Pclass': 'pclass', 
                                                  'Name': 'name', 
                                                  'Sex': 'sex', 
                                                  'Age': 'age', 
                                                  'SibSp': 'sibsp',
                                                  'Parch': 'parch', 
                                                  'Ticket': 'ticket',
                                                  'Fare': 'fare',                                                   
                                                  'Embarked': 'embarked'}, inplace = False)

train_original = train_original.drop(['PassengerId','Cabin'], axis=1)
test_original = test_original.drop(['Cabin'], axis=1)

In [5]:
train_original.info()
test_original.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   survived  891 non-null    int64  
 1   pclass    891 non-null    int64  
 2   name      891 non-null    object 
 3   sex       891 non-null    object 
 4   age       714 non-null    float64
 5   sibsp     891 non-null    int64  
 6   parch     891 non-null    int64  
 7   ticket    891 non-null    object 
 8   fare      891 non-null    float64
 9   embarked  889 non-null    object 
dtypes: float64(2), int64(4), object(4)
memory usage: 69.7+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   pclass       418 non-null    int64  
 2   name         418 non-null    object 
 3   sex          418 non-null    object 
 4   ag

In [6]:
title_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5}
def organize_titles(dataset):
  dataset['title'] = dataset.name.str.extract(' ([A-Za-z]+)\.', expand=False)
  dataset['title'] = dataset['title'].map(title_mapping)
  dataset['title'] = dataset['title'].fillna(0)

organize_titles(train_original)
organize_titles(test_original)
train_original = train_original.drop(['name'], axis=1)
test_original = test_original.drop(['name'], axis=1)
train_original.head()

,survived,pclass,sex,age,sibsp,parch,ticket,fare,embarked,title
0,0,3,male,22.0,1,0,A/5 21171,7.2500,S,1.0
1,1,1,female,38.0,1,0,PC 17599,71.2833,C,3.0
2,1,3,female,26.0,0,0,STON/O2. 3101282,7.9250,S,2.0
3,1,1,female,35.0,1,0,113803,53.1000,S,3.0
4,0,3,male,35.0,0,0,373450,8.0500,S,1.0


In [7]:
train_original['sex'] = train_original['sex'].map( {'female': 1, 'male': 0} ).astype(int)
test_original['sex'] = test_original['sex'].map( {'female': 1, 'male': 0} ).astype(int)

In [8]:
def age_fill(dataset):
  guess_ages = np.zeros((2,3))
  for i in range(0, 2):
    for j in range(0, 3):
      guess_df = dataset[(dataset['sex'] == i) & (dataset['pclass'] == j+1)]['age'].dropna()
      age_guess = guess_df.median()
      guess_ages[i,j] = int( age_guess/0.5 + 0.5 ) * 0.5
        
  for i in range(0, 2):
    for j in range(0, 3):
      dataset.loc[ (dataset.age.isnull()) & (dataset.sex == i) & (dataset.pclass == j+1),'age'] = guess_ages[i,j]
  dataset['age'] = dataset['age'].astype(int)

def age_organize(dataset):
  dataset.loc[ dataset['age'] <= 16, 'age'] = 0
  dataset.loc[(dataset['age'] > 16) & (dataset['age'] <= 32), 'age'] = 1
  dataset.loc[(dataset['age'] > 32) & (dataset['age'] <= 48), 'age'] = 2
  dataset.loc[(dataset['age'] > 48) & (dataset['age'] <= 64), 'age'] = 3
  dataset.loc[ dataset['age'] > 64, 'age'] = 4
  dataset['age*class'] = dataset.age * dataset.pclass

age_fill(train_original)
age_organize(train_original)

age_fill(test_original)
age_organize(test_original)

train_original.info()
test_original.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 11 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   survived   891 non-null    int64  
 1   pclass     891 non-null    int64  
 2   sex        891 non-null    int64  
 3   age        891 non-null    int64  
 4   sibsp      891 non-null    int64  
 5   parch      891 non-null    int64  
 6   ticket     891 non-null    object 
 7   fare       891 non-null    float64
 8   embarked   889 non-null    object 
 9   title      891 non-null    float64
 10  age*class  891 non-null    int64  
dtypes: float64(2), int64(7), object(2)
memory usage: 76.7+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   pclass       418 non-null    int64  
 2   sex          418 non-null    int64

In [9]:
train_original['familysize'] = train_original['sibsp'] + train_original['parch'] + 1
test_original['familysize'] = test_original['sibsp'] + test_original['parch'] + 1

def familly(dataset):
    dataset['isalone'] = 0
    dataset.loc[dataset['familysize'] == 1, 'isalone'] = 1

familly(train_original)
familly(test_original)
train_original = train_original.drop(['sibsp', 'parch', 'familysize'], axis=1)
test_original = test_original.drop(['sibsp', 'parch', 'familysize'], axis=1)
train_original.head()

,survived,pclass,sex,age,ticket,fare,embarked,title,age*class,isalone
0,0,3,0,1,A/5 21171,7.2500,S,1.0,3,0
1,1,1,1,2,PC 17599,71.2833,C,3.0,2,0
2,1,3,1,1,STON/O2. 3101282,7.9250,S,2.0,3,1
3,1,1,1,2,113803,53.1000,S,3.0,2,0
4,0,3,0,2,373450,8.0500,S,1.0,6,1


In [10]:
def embark(dataset, freq_port):
  dataset['embarked'] = dataset['embarked'].fillna(freq_port)
  dataset['embarked'] = dataset['embarked'].map( {'S': 0, 'C': 1, 'Q': 2} ).astype(int)

freq_port = train_original.embarked.dropna().mode()[0]
embark(train_original, freq_port)
embark(test_original, freq_port)
train_original.head()

,survived,pclass,sex,age,ticket,fare,embarked,title,age*class,isalone
0,0,3,0,1,A/5 21171,7.2500,0,1.0,3,0
1,1,1,1,2,PC 17599,71.2833,1,3.0,2,0
2,1,3,1,1,STON/O2. 3101282,7.9250,0,2.0,3,1
3,1,1,1,2,113803,53.1000,0,3.0,2,0
4,0,3,0,2,373450,8.0500,0,1.0,6,1


In [11]:
def fare_organize(dataset):
  dataset.loc[ dataset['fare'] <= 7.91, 'fare'] = 0
  dataset.loc[(dataset['fare'] > 7.91) & (dataset['fare'] <= 14.454), 'fare'] = 1
  dataset.loc[(dataset['fare'] > 14.454) & (dataset['fare'] <= 31), 'fare']   = 2
  dataset.loc[ dataset['fare'] > 31, 'fare'] = 3
 

train_original['fare'].fillna(train_original['fare'].dropna().median(), inplace=True)
test_original['fare'].fillna(test_original['fare'].dropna().median(), inplace=True)
fare_organize(train_original)
fare_organize(test_original)
train_original.head()

,survived,pclass,sex,age,ticket,fare,embarked,title,age*class,isalone
0,0,3,0,1,A/5 21171,0.0,0,1.0,3,0
1,1,1,1,2,PC 17599,3.0,1,3.0,2,0
2,1,3,1,1,STON/O2. 3101282,1.0,0,2.0,3,1
3,1,1,1,2,113803,3.0,0,3.0,2,0
4,0,3,0,2,373450,1.0,0,1.0,6,1


In [12]:
train_original = train_original.drop(['ticket'], axis=1)
test_original = test_original.drop(['ticket'], axis=1)
train_original.head()

,survived,pclass,sex,age,fare,embarked,title,age*class,isalone
0,0,3,0,1,0.0,0,1.0,3,0
1,1,1,1,2,3.0,1,3.0,2,0
2,1,3,1,1,1.0,0,2.0,3,1
3,1,1,1,2,3.0,0,3.0,2,0
4,0,3,0,2,1.0,0,1.0,6,1


In [13]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression, Perceptron, SGDClassifier
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, BaggingClassifier, RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.naive_bayes import GaussianNB, MultinomialNB

from sklearn.model_selection import cross_validate
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RepeatedStratifiedKFold

from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV

from scipy.stats import uniform

In [18]:
train_set = train_original.copy()
test_set = test_original.copy()

In [19]:
targets = list(train_set['survived'])
train_set.drop(['survived'], axis = 1, inplace = True)
train_set.head()

,pclass,sex,age,fare,embarked,title,age*class,isalone
0,3,0,1,0.0,0,1.0,3,0
1,1,1,2,3.0,1,3.0,2,0
2,3,1,1,1.0,0,2.0,3,1
3,1,1,2,3.0,0,3.0,2,0
4,3,0,2,1.0,0,1.0,6,1


In [21]:
def train_with_grid(modelo, parametros, sc, validacao):
  print('Tunnig with grid search')
  grid_search = GridSearchCV(modelo, 
                              param_grid = parametros,
                              scoring = sc, 
                              cv = validacao, 
                              verbose = 1, 
                              n_jobs = -1)

  return grid_search

def train_with_random_search(modelo, parametros, sc, validacao):
  n_iter = 30
  print('Tunnig with random search')
  randomized_search = RandomizedSearchCV(modelo, 
                                           param_distributions = parametros,
                                           scoring = sc, 
                                           cv = validacao,
                                           n_iter = n_iter, 
                                           verbose = 1, 
                                           n_jobs = -1)
  return randomized_search

def train_with_tunning(modelo, parametros, validacao, treino, targets, random_tunnig=True, sc='balanced_accuracy'):
    
    if (random_tunnig):
      model_tunned = train_with_random_search(modelo, parametros, sc, validacao)
    else:
      model_tunned = train_with_grid(modelo, parametros, sc, validacao)
    model_tunned.fit(treino, targets) 
    
    print('Acuracia: {:.4f}'.format(model_tunned.best_score_))
    print('Melhores parâmetros: {}'.format(model_tunned.best_params_))
    return model_tunned

In [22]:
log_regression_parameters = dict(C=uniform(loc=0, scale=4), penalty=['l1', 'l2'])

sgd_parameters = {
    'average': [True, False]
    }

knn_parameters = {
    'n_neighbors' : np.arange(1, 50, 2)
    }

tree_parameters = {
    'max_depth' : np.arange(1,11), 
    'min_samples_split': np.arange(2,12),
    'min_samples_leaf': np.arange(1,11),
    }

rand_forest_parameters = {   
    'max_depth' : [14, 16, 18, 20],
    'n_estimators': [140, 150, 160],
    'max_features': ['sqrt', 'auto', 'log2'],
    'min_samples_split': [7, 9, 11],
    'min_samples_leaf': [1, 2],
    'bootstrap': [True, False]
    }

In [23]:
all_predictions = []
all_predictions.append({'name':'knn', 'model': KNeighborsClassifier(), 'parameters':knn_parameters, 'score': 0.0,'best_parameters': None, 'predictor': None})
all_predictions.append({'name':'tree', 'model': DecisionTreeClassifier(),  'parameters':tree_parameters,  'score': 0.0, 'best_parameters': None, 'predictor': None})
all_predictions.append({'name':'random forest', 'model': RandomForestClassifier(), 'parameters':rand_forest_parameters, 'score': 0.0,'best_parameters': None,'predictor': None})
all_predictions.append({'name':'SGDClassifier', 'model': SGDClassifier(), 'parameters':sgd_parameters, 'score': 0.0,'best_parameters': None,'predictor': None})
all_predictions.append({'name':'Logistic Regression', 'model': LogisticRegression(), 'parameters':log_regression_parameters, 'score': 0.0,'best_parameters': None,'predictor': None})
all_predictions.append({'name':'SVC', 'model': SVC(), 'parameters':{}, 'score': 0.0,'best_parameters': None,'predictor': None})
all_predictions.append({'name':'LinearSVC', 'model': LinearSVC(), 'parameters':{}, 'score': 0.0,'best_parameters': None,'predictor': None})
all_predictions.append({'name':'XGBClassifier', 'model': XGBClassifier(), 'parameters':{}, 'score': 0.0,'best_parameters': None,'predictor': None})
all_predictions.append({'name':'GaussianNB', 'model': GaussianNB(), 'parameters':{}, 'score': 0.0,'best_parameters': None,'predictor': None})
all_predictions.append({'name':'AdaBoostClassifier', 'model': AdaBoostClassifier(), 'parameters':{}, 'score': 0.0,'best_parameters': None,'predictor': None})
all_predictions.append({'name':'GradientBoostingClassifier', 'model': GradientBoostingClassifier(), 'parameters':{}, 'score': 0.0,'best_parameters': None,'predictor': None})
all_predictions.append({'name':'Perceptron', 'model': Perceptron(), 'parameters':{}, 'score': 0.0,'best_parameters': None,'predictor': None})
all_predictions.append({'name':'BaggingClassifier', 'model': BaggingClassifier(), 'parameters':{}, 'score': 0.0,'best_parameters': None,'predictor': None})

In [25]:
import time

## ['accuracy', 'recall', 'precision', 'balanced_accuracy', 'f1']
sc = 'accuracy'
cv = RepeatedStratifiedKFold(n_splits = 10, n_repeats = 10)
random_tunnig = True

for predictor in all_predictions:
  initial_time = time.time()

  print('Name: {}'.format(predictor['name']))

  result = train_with_tunning(predictor['model'], 
                              predictor['parameters'], 
                              cv, 
                              train_set, 
                              targets, 
                              random_tunnig, 
                              sc)
  
  predictor['score'] = result.best_score_
  predictor['best_parameters'] = result.best_params_
  predictor['predictor'] = result
  end_time = time.time()
  print('tempo gasto: {}s'.format(end_time - initial_time))
  print('*'*100)

Name: knn
Tunnig with random search
Fitting 100 folds for each of 25 candidates, totalling 2500 fits


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:281: UserWarning: The total space of parameters 25 is smaller than n_iter=30. Running 25 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done 330 tasks      | elapsed:    4.0s
[Parallel(n_jobs=-1)]: Done 2490 tasks      | elapsed:   20.4s
[Parallel(n_jobs=-1)]: Done 2500 out of 2500 | elapsed:   20.5s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


Acuracia: 0.8091
Melhores parâmetros: {'n_neighbors': 5}
tempo gasto: 20.53113317489624s
****************************************************************************************************
Name: tree
Tunnig with random search
Fitting 100 folds for each of 30 candidates, totalling 3000 fits


[Parallel(n_jobs=-1)]: Done 956 tasks      | elapsed:    3.4s
[Parallel(n_jobs=-1)]: Done 2997 out of 3000 | elapsed:   10.4s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done 3000 out of 3000 | elapsed:   10.4s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


Acuracia: 0.8101
Melhores parâmetros: {'min_samples_split': 6, 'min_samples_leaf': 3, 'max_depth': 9}
tempo gasto: 10.491689443588257s
****************************************************************************************************
Name: random forest
Tunnig with random search
Fitting 100 folds for each of 30 candidates, totalling 3000 fits


...

[Parallel(n_jobs=-1)]: Done  88 tasks      | elapsed:    7.3s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    8.2s finished
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:281: UserWarning: The total space of parameters 1 is smaller than n_iter=30. Running 1 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


Acuracia: 0.8146
Melhores parâmetros: {}
tempo gasto: 8.321455240249634s
****************************************************************************************************
Name: Perceptron
Tunnig with random search
Fitting 100 folds for each of 1 candidates, totalling 100 fits


[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    0.5s finished
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:281: UserWarning: The total space of parameters 1 is smaller than n_iter=30. Running 1 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


Acuracia: 0.7419
Melhores parâmetros: {}
tempo gasto: 0.5090768337249756s
****************************************************************************************************
Name: BaggingClassifier
Tunnig with random search
Fitting 100 folds for each of 1 candidates, totalling 100 fits
Acuracia: 0.8114
Melhores parâmetros: {}
tempo gasto: 2.115589141845703s
****************************************************************************************************


[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    2.1s finished


In [26]:
models_results = pd.DataFrame(all_predictions)
models_results = models_results.sort_values(by='score', ascending=False)
models_results

,name,model,parameters,score,best_parameters,predictor
2,random forest,"RandomForestClassifier(bootstrap=True, ccp_alp...","{'max_depth': [14, 16, 18, 20], 'n_estimators'...",0.816860,"{'n_estimators': 140, 'min_samples_split': 7, ...",RandomizedSearchCV(cv=RepeatedStratifiedKFold(...
7,XGBClassifier,"XGBClassifier(base_score=0.5, booster='gbtree'...",{},0.814824,{},RandomizedSearchCV(cv=RepeatedStratifiedKFold(...
10,GradientBoostingClassifier,"GradientBoostingClassifier(ccp_alpha=0.0, crit...",{},0.814578,{},RandomizedSearchCV(cv=RepeatedStratifiedKFold(...
5,SVC,"SVC(C=1.0, break_ties=False, cache_size=200, c...",{},0.812688,{},RandomizedSearchCV(cv=RepeatedStratifiedKFold(...
12,BaggingClassifier,"BaggingClassifier(base_estimator=None, bootstr...",{},0.811436,{},RandomizedSearchCV(cv=RepeatedStratifiedKFold(...
1,tree,"DecisionTreeClassifier(ccp_alpha=0.0, class_we...","{'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],...",0.810099,"{'min_samples_split': 6, 'min_samples_leaf': 3...",RandomizedSearchCV(cv=RepeatedStratifiedKFold(...
3,SGDClassifier,"SGDClassifier(alpha=0.0001, average=False, cla...","{'average': [True, False]}",0.809111,{'average': True},RandomizedSearchCV(cv=RepeatedStratifiedKFold(...
0,knn,"KNeighborsClassifier(algorithm='auto', leaf_si...","{'n_neighbors': [1, 3, 5, 7, 9, 11, 13, 15, 17...",0.809090,{'n_neighbors': 5},RandomizedSearchCV(cv=RepeatedStratifiedKFold(...
4,Logistic Regression,"LogisticRegression(C=1.0, class_weight=None, d...",{'C': <scipy.stats._distn_infrastructure.rv_fr...,0.807784,"{'C': 0.5042850493218114, 'penalty': 'l2'}",RandomizedSearchCV(cv=RepeatedStratifiedKFold(...
9,AdaBoostClassifier,"AdaBoostClassifier(algorithm='SAMME.R', base_e...",{},0.807297,{},RandomizedSearchCV(cv=RepeatedStratifiedKFold(...
